<a href="https://colab.research.google.com/github/uvamsi76/Malaria_detection_from_blood_sample/blob/main/malariadetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

In [ ]:
ds,ds_info=tfds.load('malaria',with_info=True,as_supervised=True,shuffle_files=True,split=['train'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incomplete7I1GQK/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# list(ds[0].as_numpy_iterator())

In [ ]:
for data in ds[0].take(1):
    print(data)

(<tf.Tensor: shape=(103, 103, 3), dtype=uint8, numpy=
array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [ ]:
ds_info

tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_dir=PosixGPath('/tmp/tmpyoxvi2n0tfds'),
    file_format=tfrecord,
    download_size=337.08 MiB,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=27558, num_shards=4>,
    },
    citation="""@article{rajaraman2018pre,
      title={Pre-trained convolutional neural networks as feature extractors toward
      improved malaria parasite detection in thin blood smear images},


In [ ]:
trainRatio=0.8
valRatio=0.1
testRatio=0.1

In [ ]:
ds_len=len(ds[0])
ds_len

27558

In [ ]:
# train_ds=ds[0].take(int(trainRatio*ds_len))
# test_ds=ds[0].skip(int(trainRatio*ds_len))
# val_ds=test_ds.take(int(valRatio*ds_len))
# test_ds=test_ds.skip(int(valRatio*ds_len))

In [ ]:
def splits(ds,trainRatio,valRatio,testRatio):
    ds_len=len(ds[0])
    train_ds=ds[0].take(int(trainRatio*ds_len))
    test_ds=ds[0].skip(int(trainRatio*ds_len))
    val_ds=test_ds.take(int(valRatio*ds_len))
    test_ds=test_ds.skip(int(valRatio*ds_len))
    return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds=splits(ds,trainRatio,valRatio,testRatio)
# for a in train_ds.take(1):
    # print(a)

In [ ]:
list(train_ds.as_numpy_iterator())
print(1)

1


In [ ]:
def resizeing(image,label):
    return tf.image.resize(image,(224,224)),label

In [ ]:
train_ds=train_ds.map(resizeing)
val_ds=val_ds.map(resizeing)

In [ ]:
train_ds=train_ds.shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds=val_ds.shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
IM_size=224
lenet_model=tf.keras.Sequential([
    InputLayer(input_shape=(IM_size,IM_size,3)),

    Conv2D(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),

    Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),

    Flatten(),

    Dense(100,activation='relu'),
    BatchNormalization(),
    Dense(10,activation='relu'),
    BatchNormalization(),
    Dense(1,activation='sigmoid')
])

In [ ]:
lenet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 6)       168       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 16)      64        
 chNormalization)                                                
                                                        

In [ ]:
lenet_model.compile(optimizer=Adam(learning_rate=0.01),
              loss=BinaryCrossentropy(),
              metrics="accuracy"
              )

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
history=lenet_model.fit(train_ds,epochs=50,verbose=1)

Epoch 1/50
689/689 [==============================] - 53s 61ms/step - loss: 0.3505 - accuracy: 0.8498
Epoch 2/50
689/689 [==============================] - 29s 42ms/step - loss: 0.2064 - accuracy: 0.9303
Epoch 3/50
689/689 [==============================] - 33s 47ms/step - loss: 0.1862 - accuracy: 0.9380
Epoch 4/50
689/689 [==============================] - 28s 41ms/step - loss: 0.1740 - accuracy: 0.9448
Epoch 5/50
689/689 [==============================] - 30s 43ms/step - loss: 0.1642 - accuracy: 0.9466
Epoch 6/50
689/689 [==============================] - 28s 41ms/step - loss: 0.1448 - accuracy: 0.9519
Epoch 7/50
689/689 [==============================] - 28s 41ms/step - loss: 0.1380 - accuracy: 0.9548
Epoch 8/50
689/689 [==============================] - 28s 41ms/step - loss: 0.1283 - accuracy: 0.9581
Epoch 9/50
689/689 [==============================] - 30s 43ms/step - loss: 0.1180 - accuracy: 0.9609
Epoch 10/50
689/689 [==============================] - 28s 41ms/step - loss: 0.106

In [ ]:
test_ds

<_SkipDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
test_ds1=test_ds.batch(1)
test_ds1

<_BatchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
lenet_model.save('Lenetmodel')

In [ ]:
funcinput=Input(shape=(IM_size,IM_size,3), name='imput image')
x= Conv2D(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu')(funcinput)
x= BatchNormalization()(x)
x=MaxPool2D(pool_size=2,strides=2)(x)
x=Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu')(x)
x=BatchNormalization()(x)
x=MaxPool2D(pool_size=2,strides=2)(x)
x=Flatten()(x)
x=Dense(100,activation='relu')(x)
x=BatchNormalization()(x)
x=Dense(10,activation='relu')(x)
x=BatchNormalization()(x)
funcoutput=Dense(1,activation='sigmoid')(x)
lenet_model=Model(funcinput,funcoutput,name='Lenetmodel')
lenet_model.summary()

Model: "Lenetmodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 imput image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 222, 222, 6)       168       
                                                                 
 batch_normalization_4 (Bat  (None, 222, 222, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 111, 111, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_5 (Bat  (None, 109, 109, 16)      6